# Conversation summarization

In long-running conversations with AI agents, the context window quickly fills with message history, leaving less room for relevant information and task instructions. As conversations extend over dozens or hundreds of turns, the agent must maintain awareness of past interactions while staying within token limits. Without proper management, older messages consume valuable context space, forcing the system to either truncate important history or exceed token budgets.

Conversation summarization addresses this challenge by periodically condensing older conversation turns into compact summaries that preserve essential information while dramatically reducing token count. Rather than keeping every message verbatim, the system identifies conversation segments that can be compressed, applies LLM-based summarization with explicit preservation instructions, and replaces the original turns with concise summaries. This maintains conversation continuity and context awareness while freeing space for new interactions and relevant information.

This notebook demonstrates how to implement conversation summarization from basic approaches to production-ready systems. We will explore simple message window truncation, progressive summarization strategies, selective information preservation, and complete conversation memory managers that balance verbatim recent history with summarized older context. The techniques shown here are essential for building agents that maintain coherent, context-aware interactions across extended conversation sessions.

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any
from datetime import datetime
import os
from dotenv import load_dotenv

### Initialize the language model for summarization and conversation

In [2]:
# Using gpt-4o-mini for cost-effective summarization
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY", "").strip(), temperature=0)

We use `temperature=0.0` to ensure our agent produces consistent summaries.

## Basic conversation truncation
When conversations grow beyond a certain length, the simplest survival strategy is to just cut off the old parts and keep only what is recent. Think of it like a sliding window that moves forward with each new message, dropping whatever falls outside the frame. This approach is appealingly straightforward - no complex logic, no LLM calls, just straightforward list slicing. But simplicity comes at a cost. When we truncate, we are not selectively forgetting less important details; we are indiscriminately erasing entire chunks of history.

This naive truncation strategy serves an important purpose in our learning journey though. It establishes a baseline that helps us understand why more sophisticated approaches are necessary. In production systems where agents need to maintain coherent conversations over dozens or hundreds of turns, remembering that a customer has been promised expedited shipping or that their order number is #12345 is not optional - it is essential for maintaining trust and delivering quality service. We will implement this basic truncation to see exactly how much context gets lost and why that matters, setting the stage for understanding the value of proper summarization techniques.

In [3]:
def truncate_conversation(messages: List[Any], max_messages: int = 10) -> List[Any]:
    """
    Simple truncation keeping only the most recent messages.
    
    Args:
        messages: List of conversation messages
        max_messages: Maximum number of messages to retain
        
    Returns:
        Truncated list of messages
    """
    # Keep only the last N messages to stay within limits
    if len(messages) <= max_messages:
        return messages
    
    # Separate system messages from conversation history
    system_messages = [msg for msg in messages if isinstance(msg, SystemMessage)]  # System messages contain instructions and should typically be preserved
    conversation_messages = [msg for msg in messages if not isinstance(msg, SystemMessage)]

    # Calculate how many conversation messages we can keep
    available_slots = max_messages - len(system_messages)
    # Keep system messages and truncate conversation history
    truncated_conversation = conversation_messages[-available_slots:]

    # Return system messages first, then truncated conversation history
    return system_messages + truncated_conversation

The truncation function implements a straightforward sliding window approach by first separating system messages from conversation turns, ensuring that important system instructions are always preserved regardless of truncation limits. It then calculates how many conversation messages can fit in the remaining space after accounting for system messages, using negative list slicing to grab only the most recent turns that fit within the budget. System messages are distinguished using isinstance checks and are always placed at the beginning of the returned list to maintain proper message ordering for LLM consumption.

In [4]:
# Create a simulated customer service conversation to demonstrate truncation
# This represents a typical support interaction with multiple back-and-forth turns
long_conversation = [
    SystemMessage(content="You are a helpful customer service assistant."),
    HumanMessage(content="I need help with my order #12345"),
    AIMessage(content="I'd be happy to help! Let me look up order #12345."),
    HumanMessage(content="It was supposed to arrive yesterday"),
    AIMessage(content="I see it's delayed. Let me check the shipping status."),
    HumanMessage(content="I need it urgently for a gift"),
    AIMessage(content="I understand the urgency. I can offer expedited shipping."),
    HumanMessage(content="How much will that cost?"),
    AIMessage(content="Expedited shipping is $15, but I can waive the fee."),
    HumanMessage(content="That would be great, thank you!"),
    AIMessage(content="Done! Your order will arrive tomorrow by 5 PM."),
    HumanMessage(content="Actually, can I change the delivery address?"),
]

print(f"Created conversation with {len(long_conversation)} messages")
print(f"Includes {sum(1 for msg in long_conversation if isinstance(msg, SystemMessage))} system message(s)")
print(f"And {sum(1 for msg in long_conversation if not isinstance(msg, SystemMessage))} conversation turns")

# Apply truncation to keep only recent messages
truncated = truncate_conversation(long_conversation, max_messages=6)

print("\nTruncated Conversation:")
print("="*80)
for i, msg in enumerate(truncated, 1):
    msg_type = type(msg).__name__
    print(f"{i}. {msg_type}: {msg.content}")

print(f"\n{'='*80}")
print(f"\nOriginal: {len(long_conversation)} messages")
print(f"Truncated: {len(truncated)} messages")
print(f"Lost: {len(long_conversation) - len(truncated)} messages")

Created conversation with 12 messages
Includes 1 system message(s)
And 11 conversation turns

Truncated Conversation:
1. SystemMessage: You are a helpful customer service assistant.
2. HumanMessage: How much will that cost?
3. AIMessage: Expedited shipping is $15, but I can waive the fee.
4. HumanMessage: That would be great, thank you!
5. AIMessage: Done! Your order will arrive tomorrow by 5 PM.
6. HumanMessage: Actually, can I change the delivery address?


Original: 12 messages
Truncated: 6 messages
Lost: 6 messages


In our demonstration, the 12-message conversation gets truncated to just 6 messages total, which means the agent loses visibility into the first 6 conversation turns. This includes the order number #12345 that was mentioned at the start, the entire context about why the delivery is urgent (it's a gift), and the commitment to expedited shipping with a waived fee. The agent now only sees the tail end of the conversation starting from around the fee discussion, making it impossible to provide contextually aware responses about the order details or honor the commitments made earlier. This dramatic information loss demonstrates why naive truncation is insufficient for production systems that need to maintain conversation continuity and customer trust.

## LLM-based summarization
Instead of throwing away old messages, what if we could distill them down to their essence? This is where LLM-based summarization transforms our approach from crude amputation to intelligent compression. The language model becomes a skilled editor, reading through conversation turns and extracting what truly matters while letting the conversational fluff fall away. When we ask the LLM to summarize, we are not leaving it to guess what is important - we provide explicit preservation instructions that guide it toward the information that has lasting value across the conversation lifecycle.

The key insight here is that not all information ages equally. Order numbers, customer names, specific problems raised, solutions promised, and pending actions all have enduring relevance that transcends the immediate conversational turn. Meanwhile, the social niceties and transitional phrases that make conversations feel natural lose their value once the moment passes. A well-crafted summarization prompt instructs the model to identify and preserve these high-value information categories while condensing the rest into concise narrative form. This creates summaries that are typically 60-80% shorter than the original messages while maintaining all the critical context an agent needs to provide coherent, informed responses. The result is conversation histories that can extend much further within the same context budget, enabling longer, more complex interactions without sacrificing continuity or overwhelming the model with irrelevant historical detail.

In [5]:
def summarize_messages(messages: List[Any], llm: ChatOpenAI) -> str:
    """
    Use LLM to create a concise summary of conversation messages.
    
    Args:
        messages: List of messages to summarize
        llm: Language model for generating summaries
        
    Returns:
        Concise summary string preserving key information
    """
    # Convert message objects into a readable text format for the LLM - Each line shows the message type (HumanMessage/AIMessage) and content
    conversation_text = "\n".join([
        f"{type(msg).__name__}: {msg.content}" 
        for msg in messages
    ])
    
    # Create a detailed summarization prompt with explicit preservation instructions
    summarization_prompt = f"""Summarize the following conversation concisely in 1-2 sentences.

Preserve these key details:
- Order/ticket numbers and IDs
- Main problem or request
- Solution provided
- Any commitments made

Conversation:
{conversation_text}

Write a brief summary (1-2 sentences):"""
    
    # Invoke the LLM to generate the summary based on our instructions
    response = llm.invoke([HumanMessage(content=summarization_prompt)])

    # Return just the text content of the summary
    return response.content

The summarization function transforms message objects into formatted text that the LLM can process, then constructs a carefully designed prompt that explicitly lists the types of information that must survive compression. This prompt engineering is critical - without clear preservation instructions, the LLM might focus on narrative flow while accidentally dropping crucial identifiers or commitments. By specifying exactly what matters (identifiers, problems, solutions, status), we guide the model toward consistent, reliable summarization behavior. The `conversation_text` formatting uses `type(msg).name` to include message type information, helping the LLM understand the conversational structure when generating summaries.

In [6]:
# Example: Summarize the early part of the conversation
messages_to_summarize = long_conversation[1:7]  # Skip system message, summarize first 6 turns

summary = summarize_messages(messages_to_summarize, llm)

print("Original Messages:")
print("="*80)
for msg in messages_to_summarize:
    print(f"{type(msg).__name__}: {msg.content}")

print(f"\n{'='*80}")
print("\nGenerated Summary:")
print("-" * 80)
print(summary)

# Calculate token savings (approximate)
original_chars = sum(len(msg.content) for msg in messages_to_summarize)
summary_chars = len(summary)
compression_ratio = (1 - summary_chars / original_chars) * 100

print(f"\n{'='*80}")
print(f"  Original characters: {original_chars}")
print(f"  Summary characters: {summary_chars}")
print(f"  Compression: {compression_ratio:.1f}%")
print(f"  Estimated token savings: ~{int((original_chars - summary_chars) / 4)} tokens")

Original Messages:
HumanMessage: I need help with my order #12345
AIMessage: I'd be happy to help! Let me look up order #12345.
HumanMessage: It was supposed to arrive yesterday
AIMessage: I see it's delayed. Let me check the shipping status.
HumanMessage: I need it urgently for a gift
AIMessage: I understand the urgency. I can offer expedited shipping.


Generated Summary:
--------------------------------------------------------------------------------
The customer inquired about order #12345, which was delayed and was supposed to arrive yesterday, expressing urgency as it was a gift. The AI offered a solution of expedited shipping to address the issue.

  Original characters: 256
  Summary characters: 204
  Compression: 20.3%
  Estimated token savings: ~13 tokens


The example demonstrates typical compression ratios of 20-60%, and it varies depending on how verbose the original messages were. This reduction in token count allows systems to maintain much longer conversation histories within the same context window.

## Progressive summarization strategy
In production systems, we do not want to summarize all messages at once or wait until the conversation is too long. Instead, we implement a progressive strategy that periodically summarizes older message segments while keeping recent messages verbatim. This creates a layered conversation history where recent turns remain detailed and immediately accessible, while older turns exist as increasingly compressed summaries. The agent maintains both immediate context and historical awareness without exponential context growth.

This progressive approach typically divides conversation history into segments: recent messages kept verbatim, middle-aged messages that get summarized into a rolling summary, and very old history that may be further compressed or stored externally. We will implement a system that automatically triggers summarization when the conversation exceeds thresholds, replacing old message batches with their summaries to maintain a stable context size.

In [7]:
class ConversationSummarizer:
    """
    Manages conversation history with progressive summarization.
    Keeps recent messages verbatim, older messages as summaries.
    """
    
    def __init__(
        self, 
        llm: ChatOpenAI,
        max_recent_messages: int = 10,
        summarize_batch_size: int = 6
    ):
        """
        Initialize the conversation summarizer.
        
        Args:
            llm: Language model for summarization
            max_recent_messages: Number of recent messages to keep verbatim
            summarize_batch_size: Number of messages to include in each summary
        """
        # Store the LLM instance for generating summaries when needed
        self.llm = llm
        # Configuration parameters for managing the layered memory
        self.max_recent_messages = max_recent_messages  # Threshold for triggering summarization
        self.summarize_batch_size = summarize_batch_size  # How many old messages to compress at once
        
        # Storage for different types of conversation content
        self.system_messages = []  # System instructions (always preserved)
        self.summaries = []  # Summaries of older conversation segments
        self.recent_messages = []  # Recent messages kept in full detail
    
    def add_message(self, message: Any) -> None:
        """
        Add a new message and trigger summarization if needed.
        
        Args:
            message: New message to add to conversation history
        """
        # Separate system messages from conversation messages
        if isinstance(message, SystemMessage):
            self.system_messages.append(message)  # System messages go into their own preserved list - they never get summarized because they define agent behavior
            return
        
        # Regular conversation messages go into the recent buffer
        self.recent_messages.append(message)
        
        # Check if the recent buffer has exceeded our threshold. If so, trigger automatic summarization to compress old messages
        if len(self.recent_messages) > self.max_recent_messages:
            self._summarize_old_messages()
    
    def _summarize_old_messages(self) -> None:
        """
        Internal method to summarize a batch of older messages.
        Moves oldest messages from recent_messages to summaries.
        """
        # Extract the oldest batch of messages to summarize - these are messages that have "aged out" of the detailed view
        messages_to_summarize = self.recent_messages[:self.summarize_batch_size]
        
        # Generate summary of this batch using our LLM summarization function
        summary_text = summarize_messages(messages_to_summarize, self.llm)
        
        # Create a summary message with metadata for tracking - the additional_kwargs stores useful information about the summary
        summary_message = AIMessage(
            content=f"[CONVERSATION SUMMARY]: {summary_text}",
            additional_kwargs={
                "is_summary": True,  # Flag to identify summary messages
                "summarized_count": len(messages_to_summarize),  # How many messages were compressed
                "timestamp": datetime.now().isoformat()  # When summarization occurred
            }
        )
        
        # Add summary to summaries list
        self.summaries.append(summary_message)
        
        # Remove the now-summarized messages from recent messages
        self.recent_messages = self.recent_messages[self.summarize_batch_size:]
    
    def get_messages(self) -> List[Any]:
        """
        Get the complete conversation history: system + summaries + recent.
        
        Returns:
            Combined list of all messages in chronological order
        """
        # Combine all three tiers in chronological order:
        # 1. System messages (always first)
        # 2. Summaries of old conversations
        # 3. Recent detailed messages
        return self.system_messages + self.summaries + self.recent_messages
    
    def get_stats(self) -> Dict[str, int]:
        """
        Get statistics about the conversation state.
        
        Returns:
            Dictionary with message counts and total context size
        """
        return {
            "system_messages": len(self.system_messages),
            "summaries": len(self.summaries),
            "recent_messages": len(self.recent_messages),
            "total_messages": len(self.get_messages())
        }

- The `ConversationSummarizer` class initializes with three separate storage lists to implement a layered memory architecture.
    - The `system_messages` list holds system instructions that define agent behavior and should never be compressed or removed.
    - The `summaries` list accumulates compressed representations of older conversation segments as they age out of recent memory.
    - The `recent_messages` list maintains a sliding window of verbatim messages, configured by `max_recent_messages` parameter.
    - The `summarize_batch_size` parameter controls how many old messages get compressed together in each summarization operation, typically set to 4-6 messages to create meaningful summaries without losing too much context in a single compression step.
- The `add_message method` implements the automatic triggering logic for progressive summarization.
    - It first checks if the incoming message is a `SystemMessage` using `isinstance`, routing it to the preserved `system_messages list` if so.
    - Regular conversation messages (`HumanMessage` and `AIMessage`) are appended to the `recent_messages` buffer.
    - After each addition, the method checks whether the recent buffer exceeds the `max_recent_messages` threshold, automatically calling `_summarize_old_messages` if the limit is breached.
  
  This creates a self-managing system where conversation history is automatically compressed as it grows, without requiring external intervention or manual triggers.
- The `_summarize_old_messages` method implements the core compression logic by extracting the oldest batch of messages using list slicing `[:self.summarize_batch_size]`, generating a summary via the `summarize_messages` function, and wrapping the result in an `AIMessage` with tracking metadata in `additional_kwargs`.
    - The metadata includes `is_summary` flag for easy identification, `summarized_count` to track how many messages were compressed, and a `timestamp` for audit trails.
  
  After creating the summary, the method removes the summarized messages from recent_messages using list slicing `[self.summarize_batch_size:]`, maintaining a stable buffer size.
- The `get_messages` method provides a unified view by concatenating all three lists in chronological order.
- `get_stats` returns metrics for monitoring the memory state.

In [8]:
# Example: Process the long conversation with progressive summarization
summarizer = ConversationSummarizer(llm, max_recent_messages=6, summarize_batch_size=4)

# Add all messages one by one
for msg in long_conversation:
    summarizer.add_message(msg)
    
# Display the final conversation state
final_messages = summarizer.get_messages()
stats = summarizer.get_stats()

print("Conversation Statistics:")
print(f"  System messages: {stats['system_messages']}")
print(f"  Summary segments: {stats['summaries']}")
print(f"  Recent verbatim messages: {stats['recent_messages']}")
print(f"  Total context items: {stats['total_messages']}")
print(f"\nOriginal message count: {len(long_conversation)}")
print(f"Compressed message count: {stats['total_messages']}")
print(f"Reduction: {len(long_conversation) - stats['total_messages']} messages")

print("\n" + "="*80)
print("\nFinal Conversation Context:")
for i, msg in enumerate(final_messages, 1):
    msg_type = type(msg).__name__
    is_summary = msg.additional_kwargs.get("is_summary", False) if hasattr(msg, "additional_kwargs") else False
    prefix = "[SUMMARY]" if is_summary else ""
    content_preview = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
    print(f"{i}. {prefix} {msg_type}: {content_preview}")

Conversation Statistics:
  System messages: 1
  Summary segments: 2
  Recent verbatim messages: 3
  Total context items: 6

Original message count: 12
Compressed message count: 6
Reduction: 6 messages


Final Conversation Context:
1.  SystemMessage: You are a helpful customer service assistant.
2. [SUMMARY] AIMessage: [CONVERSATION SUMMARY]: The customer inquired about order #12345, which was supposed to arrive yeste...
3. [SUMMARY] AIMessage: [CONVERSATION SUMMARY]: The customer urgently requested expedited shipping for a gift, and the AI of...
4.  HumanMessage: That would be great, thank you!
5.  AIMessage: Done! Your order will arrive tomorrow by 5 PM.
6.  HumanMessage: Actually, can I change the delivery address?


In our demonstration with a 12-message conversation, the system compressed it to 6 context items: 1 system message, 2 summaries covering the first 8 conversation turns, and 3 recent messages kept verbatim. This reduction from 12 to 6 items (50% compression) maintains complete conversation awareness while stabilizing context size. The key insight is that as conversations grow longer, the context size remains bounded - a 100-message conversation might still compress to just 10-15 context items, making the system scalable to arbitrarily long interactions without context overflow. The summary preserves critical information like order #12345 and the expedited shipping commitment, while the recent messages provide detailed context for the most current exchanges.

## Structured summary with preservation rules
For production systems handling complex conversations with multiple types of important information, we need more sophisticated summarization that uses structured formats and explicit preservation rules. Rather than generating free-form summaries, we can define schema-based summary structures that ensure specific types of information are always captured. This might include separate fields for customer identifiers, issues raised, resolutions provided, pending actions, and important dates or numbers.

Using Pydantic models, we can enforce that summaries contain required information fields, making the summaries more reliable and easier to parse programmatically. This structured approach is particularly valuable when summaries need to be referenced by downstream systems or when specific information types must be preserved for compliance or business logic reasons.

In [9]:
class ConversationSummary(BaseModel):
    """
    Structured format for conversation summaries with explicit fields.
    Ensures critical information categories are always captured.
    """
    # Required field: Identifiers and references that must be preserved
    identifiers: List[str] = Field(
        description="Important IDs, order numbers, account numbers, etc."
    )
    
    # Required field: Customer's main concerns or requests - Captures the "why" of the conversation - what brought the customer here
    key_issues: List[str] = Field(
        description="Main problems, questions, or requests raised"
    )
    
    # Required field: Actions taken or solutions provided - Documents what the agent committed to or accomplished
    resolutions: List[str] = Field(
        description="Solutions provided, actions taken, commitments made"
    )
    
    # Optional field: Outstanding items that need follow-up - Tracks unresolved issues or future actions
    pending_actions: List[str] = Field(
        default_factory=list,  # Uses default_factory to return empty list if no pending actions
        description="Unresolved issues, pending tasks, or future actions"
    )
    
    # Optional field: Important contextual details - Captures anything else that might be relevant but doesn't fit other categories
    context_notes: Optional[str] = Field(
        default=None,
        description="Additional context that may be relevant"
    )

The Pydantic model defines a rigid structure with three required fields (`identifiers`, `key_issues`, `resolutions`) and two optional fields (`pending_actions`, `context_notes`), ensuring that every summary captures the minimum essential information while allowing flexibility for additional context. Each field includes a description that serves double duty: documenting the schema for developers and guiding the LLM on what information belongs in each category. The `identifiers` field uses `List[str]` to capture multiple IDs or reference numbers, while `pending_actions` uses `default_factory=list` to provide an empty list when no pending items exist, avoiding `None` values that require additional handling.

In [10]:
def create_structured_summary(
    messages: List[Any], 
    llm: ChatOpenAI
) -> ConversationSummary:
    """
    Generate a structured summary using Pydantic model enforcement.
    
    Args:
        messages: Messages to summarize
        llm: Language model with structured output support
        
    Returns:
        Structured ConversationSummary object
    """
    # Format conversation for analysis
    conversation_text = "\n".join([
        f"{type(msg).__name__}: {msg.content}" 
        for msg in messages
    ])
    
    # Create prompt for structured summarization
    prompt = f"""Analyze the following conversation and extract structured information.

Conversation:
{conversation_text}

Extract:
- identifiers: Any order numbers, IDs, account numbers, or reference codes
- key_issues: Main problems, questions, or requests from the user
- resolutions: Solutions provided, actions taken, or commitments made
- pending_actions: Any unresolved issues or tasks that need follow-up
- context_notes: Other relevant context
"""
    
    # Use structured output to ensure schema compliance
    llm_with_structure = llm.with_structured_output(ConversationSummary)
    
    # Generate structured summary
    summary = llm_with_structure.invoke([HumanMessage(content=prompt)])
    
    return summary

The `create_structured_summary` function leverages LangChain's `with_structured_output` method, which constrains the LLM to produce output that exactly matches the Pydantic schema. This is not just validation after the fact - the structured output mechanism actually modifies how the LLM generates its response, ensuring compliance with the schema structure. The prompt provides additional guidance by explaining what each field should contain, helping the LLM understand which pieces of information map to which schema fields. The conversation formatting uses `type(msg).name` to preserve message type information, giving the LLM context about who said what when extracting key information.

In [11]:
# Example: Create structured summary of conversation segment
messages_for_structured_summary = long_conversation[1:9]

structured_summary = create_structured_summary(messages_for_structured_summary, llm)

print("Structured Conversation Summary:")
print("="*80)
print("\nIdentifiers:")
for identifier in structured_summary.identifiers:
    print(f"  - {identifier}")

print("\nKey Issues:")
for issue in structured_summary.key_issues:
    print(f"  - {issue}")

print("\nResolutions:")
for resolution in structured_summary.resolutions:
    print(f"  - {resolution}")

print("\nPending Actions:")
if structured_summary.pending_actions:
    for action in structured_summary.pending_actions:
        print(f"  - {action}")
else:
    print("  None")

if structured_summary.context_notes:
    print(f"\nContext Notes: {structured_summary.context_notes}")

Structured Conversation Summary:

Identifiers:
  - #12345

Key Issues:
  - Order was supposed to arrive yesterday
  - Need the order urgently for a gift
  - Cost of expedited shipping

Resolutions:
  - Checked the shipping status and confirmed the order is delayed
  - Offered expedited shipping and waived the fee

Pending Actions:
  None

Context Notes: The user requires the order urgently for a gift.


When we apply this to our conversation segment, we get a `ConversationSummary` object where we can reliably access `structured_summary.identifiers[0]` to get the order number, iterate through `structured_summary.resolutions` to see what was promised, and programmatically check `structured_summary.pending_actions` to identify follow-up needs. This machine-readable format enables integration with business systems - we could automatically create support tickets from `pending_actions`, log resolutions to a CRM, or validate that all interactions captured required identifiers for compliance. The structured format eliminates ambiguity and parsing complexity, making summaries first-class data objects rather than unstructured text blobs. In our example, the system correctly extracts "Order #12345" to `identifiers`, "Delayed delivery" to `key_issues`, and "Expedited shipping with waived fee" to `resolutions`.

## Production conversation memory manager
Bringing together all these techniques, we can build a production-ready conversation memory manager that handles the complete lifecycle of conversation history. This system automatically manages context size, applies progressive summarization with structured preservation, tracks token budgets, and provides interfaces for both adding new messages and retrieving optimized context for LLM calls. The manager maintains conversation quality while ensuring the agent never exceeds context limits.

This production implementation includes token estimation, automatic summarization triggers, structured summary storage, and efficient context retrieval. It provides a complete abstraction over conversation memory management, allowing application code to simply add messages without worrying about context overflow or information loss.

In [12]:
class ProductionConversationManager:
    """
    Production-ready conversation manager with automatic summarization,
    token budgeting, and structured preservation.
    """
    
    def __init__(
        self,
        llm: ChatOpenAI,
        max_context_tokens: int = 4000,
        recent_message_count: int = 10,
        summarize_batch_size: int = 6
    ):
        """
        Initialize production conversation manager.
        
        Args:
            llm: Language model for summarization and conversation
            max_context_tokens: Maximum tokens to use for conversation history
            recent_message_count: Number of recent messages to keep verbatim
            summarize_batch_size: Messages per summary batch
        """
        # Store LLM and configuration
        self.llm = llm
        self.max_context_tokens = max_context_tokens  # Hard token limit
        self.recent_message_count = recent_message_count  # Recent message threshold
        self.summarize_batch_size = summarize_batch_size  # Batch size for summarization
        
        # Three-tier conversation storage with structured summaries
        self.system_messages: List[Any] = []  # System instructions
        self.structured_summaries: List[ConversationSummary] = []  # Pydantic summary objects
        self.recent_messages: List[Any] = []  # Recent detailed messages
        
        # Metrics for monitoring and debugging
        self.total_messages_processed = 0  # Lifetime message count
        self.summarization_count = 0  # How many summarizations have occurred
    
    def _estimate_tokens(self, messages: List[Any]) -> int:
        """
        Estimate token count for a list of messages.
        Uses rough approximation of 1 token per 4 characters.
        
        Args:
            messages: Messages to estimate
            
        Returns:
            Estimated token count
        """
        # Simple approximation: ~4 chars per token for English text
        total_chars = sum(len(msg.content) for msg in messages)  # Count total characters across all message content
        return total_chars // 4  # Apply the ~4 chars per token heuristic
    
    def _check_and_summarize(self) -> None:
        """
        Check if summarization is needed and perform it.
        Triggers when recent messages exceed threshold or token budget.
        """
        # First check: Are we under the message count threshold?
        if len(self.recent_messages) <= self.recent_message_count:
            return  # If yes, no summarization needed yet
        
        # Second check: Are we within our token budget?
        all_messages = self.system_messages + self.recent_messages
        current_tokens = self._estimate_tokens(all_messages)
        
        # If we are within budget and message count, no summarization needed
        if current_tokens < self.max_context_tokens * 0.8:  # We check against 80% of max to leave headroom for response generation
            return
        
        # Both thresholds exceeded - time to summarize
        # Extract the oldest batch of messages from the recent buffer
        messages_to_summarize = self.recent_messages[:self.summarize_batch_size]
        
        # Generate a structured summary using our Pydantic model
        structured_summary = create_structured_summary(messages_to_summarize, self.llm)  # This preserves information in a reliable, machine-readable format
        
        # Store the structured summary object
        self.structured_summaries.append(structured_summary)
        
        # Remove the now-summarized messages from recent list
        self.recent_messages = self.recent_messages[self.summarize_batch_size:]
        
        # Update metrics for monitoring
        self.summarization_count += 1
    
    def add_message(self, message: Any) -> None:
        """
        Add a new message to the conversation.
        Automatically triggers summarization if thresholds are exceeded.
        
        Args:
            message: Message to add (HumanMessage, AIMessage, or SystemMessage)
        """
        # System messages go into their own preserved list
        if isinstance(message, SystemMessage):
            self.system_messages.append(message)
            return
        
        # Regular messages go into the recent buffer
        self.recent_messages.append(message)
        self.total_messages_processed += 1
        
        # Check if we need to trigger summarization - This happens automatically based on thresholds
        self._check_and_summarize()
    
    def get_context_for_llm(self) -> List[Any]:
        """
        Get optimized context for LLM calls.
        Includes system messages, summary representations, and recent messages.
        
        Returns:
            List of messages optimized for context window
        """
        context = []
        
        # Start with system messages (always first)
        context.extend(self.system_messages)
        
        # Add structured summaries converted to readable AI messages
        for summary in self.structured_summaries:
            # Format structured summary into readable text - This makes the summary understandable to the LLM
            summary_text = f"""[Previous Conversation Summary]
Identifiers: {', '.join(summary.identifiers)}
Issues: {'; '.join(summary.key_issues)}
Resolutions: {'; '.join(summary.resolutions)}
Pending: {'; '.join(summary.pending_actions) if summary.pending_actions else 'None'}"""

            # Wrap in an AIMessage so it appears as part of conversation history
            context.append(AIMessage(content=summary_text))
        
        # Finally, add recent messages in full detail
        context.extend(self.recent_messages)
        
        return context
    
    def get_stats(self) -> Dict[str, Any]:
        """
        Get comprehensive statistics about conversation state.
        
        Returns:
            Dictionary with detailed metrics
        """
        # Get current context to calculate stats
        context = self.get_context_for_llm()
        
        return {
            "total_messages_processed": self.total_messages_processed,
            "summarization_count": self.summarization_count,
            "current_context_items": len(context),
            "estimated_tokens": self._estimate_tokens(context),
            "system_messages": len(self.system_messages),
            "summaries": len(self.structured_summaries),
            "recent_messages": len(self.recent_messages),
            "token_budget_used": f"{(self._estimate_tokens(context) / self.max_context_tokens * 100):.1f}%"
        }

- The `ProductionConversationManager` initializes with comprehensive configuration parameters for token budgeting (`max_context_tokens`), recent message management (`recent_message_count`) and summarization behavior (`summarize_batch_size`).
    - The three-tier storage system uses `structured_summaries` list to hold ConversationSummary Pydantic objects rather than free-form text, ensuring reliable information preservation with machine-readable schemas.
    - The metrics tracking with `total_messages_processed` and `summarization_count` provides visibility into system behavior for monitoring dashboards and debugging.
  
  The configuration parameters provide clear knobs for tuning the system to different use cases - stricter token budgets for cost-sensitive applications or larger recent message windows for context-intensive tasks.
- The `_estimate_tokens` method implements a simple character-based heuristic (4 characters per token) that provides reasonable accuracy for English text without requiring external token counting libraries. For production systems requiring exact counts, this could be replaced with tiktoken or API-based counting.
- The `_check_and_summarize` method implements intelligent triggering logic with two threshold checks: first verifying that recent message count exceeds the limit, then checking token budget utilization against 80% of maximum to leave headroom for response generation. The 80% threshold prevents hitting hard limits during LLM calls.
    - When both thresholds are exceeded, the method triggers structured summarization using `create_structured_summary`, stores the resulting Pydantic object, removes summarized messages from the recent buffer using list slicing, and increments the `summarization_count metric`.
- The `add_message` method provides a clean public interface that handles all complexity internally. It routes system messages to the preserved list, adds regular messages to the recent buffer while incrementing `total_messages_processed`, and automatically calls `_check_and_summarize` to trigger compression when needed.
- The `get_context_for_llm` method constructs the complete context by combining system messages, formatted summaries and recent messages in chronological order. T
    - he summary formatting converts Pydantic objects into readable text with clear labels (Identifiers, Issues, Resolutions, Pending), using join operations to format lists and conditional expressions to handle empty pending_actions lists. Each formatted summary is wrapped in an AIMessage to maintain consistency with conversation message types, ensuring LLMs can process the context without special handling.

In [13]:
# Example: Use production manager with a long conversation
manager = ProductionConversationManager(
    llm=llm,
    max_context_tokens=150,
    recent_message_count=6,
    summarize_batch_size=4
)

# Process all messages
for msg in long_conversation:
    manager.add_message(msg)

# Continue with a few more messages
additional_messages = [
    HumanMessage(content="Yes, please change it to 456 Oak Street"),
    AIMessage(content="I've updated the delivery address to 456 Oak Street."),
    HumanMessage(content="Will it still arrive tomorrow?"),
    AIMessage(content="Yes, it will still arrive tomorrow by 5 PM."),
]

for msg in additional_messages:
    manager.add_message(msg)

# Get final statistics
stats = manager.get_stats()

print("Production Manager Statistics:")
print("="*80)
for key, value in stats.items():
    print(f"{key.replace('_', ' ').title()}: {value}")

print("\n" + "="*80)
print("\nOptimized Context for LLM:")
context = manager.get_context_for_llm()
for i, msg in enumerate(context, 1):
    msg_type = type(msg).__name__
    content_preview = msg.content[:150] + "..." if len(msg.content) > 150 else msg.content
    print(f"\n{i}. {msg_type}:")
    print(f"   {content_preview}")

print("\n" + "="*80)
print("\nContext Efficiency:")
original_count = len(long_conversation) + len(additional_messages)
optimized_count = len(context)
print(f"Original messages: {original_count}")
print(f"Optimized context items: {optimized_count}")
print(f"Reduction: {original_count - optimized_count} messages")
print(f"Compression ratio: {(1 - optimized_count/original_count)*100:.1f}%")

Production Manager Statistics:
Total Messages Processed: 15
Summarization Count: 2
Current Context Items: 10
Estimated Tokens: 191
System Messages: 1
Summaries: 2
Recent Messages: 7
Token Budget Used: 127.3%


Optimized Context for LLM:

1. SystemMessage:
   You are a helpful customer service assistant.

2. AIMessage:
   [Previous Conversation Summary]
Identifiers: 12345
Issues: Order was supposed to arrive yesterday; Order is delayed
Resolutions: Checked the shipping ...

3. AIMessage:
   [Previous Conversation Summary]
Identifiers: 
Issues: Need for urgent delivery for a gift; Cost of expedited shipping
Resolutions: Offered expedited s...

4. HumanMessage:
   That would be great, thank you!

5. AIMessage:
   Done! Your order will arrive tomorrow by 5 PM.

6. HumanMessage:
   Actually, can I change the delivery address?

7. HumanMessage:
   Yes, please change it to 456 Oak Street

8. AIMessage:
   I've updated the delivery address to 456 Oak Street.

9. HumanMessage:
   Will it still 

The `ProductionConversationManager` integrates all previous concepts into a complete system. It tracks token budgets using character-based estimation, automatically triggers summarization when recent message count exceeds the threshold and token budget exceeds 80% of maximum. The dual-threshold approach prevents premature summarization on short conversations while ensuring compression kicks in before hitting hard limits. 

In the example, we configure `max_context_tokens=150` to ensure summarization triggers with our 16-message conversation. The system processes messages incrementally, checking thresholds after each addition, and automatically generates structured Pydantic summaries when both conditions are met. The final statistics show how many summarization cycles occurred and the resulting compression ratio.